# Training the neural network using the Donkey library

In this chapter, you'll create and train the neural network that comes with the [Donkey](https://github.com/wroscoe/donkey) library. It is more or less the same procedure described in the [Donkey docs](http://docs.donkeycar.com/guide/train_autopilot/), only we train on a SageMaker notebook instance and not your local laptop.

## The Donkey library

The [Donkey library](https://github.com/wroscoe/donkey) has several components.

It is first and foremost a python library installed where your other python libraries are (e.g. system python or virtualenv). After installation, you can `import` it as any normal python library:

```python
import donkeycar as dk
```

It also has a CLI with tools mainly used to aid training (see [donkey-tools.ipynb](./donkey-tools.ipynb)):

```bash
donkey --help
```

A `Vehicle` application, installed to the `~/d2` directory by default. This is where you'll find the `manage.py` script, which is used for both **driving** and **training**.

```bash
~/d2/manage.py --help
```

### Install the Donkey library

If you already installed the [Donkey](https://github.com/wroscoe/donkey) library, you can [skip](#Train) this step.

Otherwise, go ahead:

In [ ]:
# Make sure we're in SageMaker root
%cd ~/SageMaker

# Remove any old versions of the library
!rm -rf ~/SageMaker/donkey

# Clone the Donkey library git
!git clone https://github.com/wroscoe/donkey.git

In [ ]:
# Update Donkey dependencies

# Keras is pinned to version 2.0.8 in the Donkey requirements. Change this to allow a newer version
!sed -i -e 's/keras==2.0.8/keras>=2.1.2/g' ~/SageMaker/donkey/setup.py
!sed -i -e 's/tensorflow>=1.1/tensorflow-gpu>=1.4/g' ~/SageMaker/donkey/setup.py

# Install
!pip uninstall --yes donkeycar
!pip install ~/SageMaker/donkey

In [ ]:
# Create a new car using the library CLI
!rm -rf ~/d2
!donkey createcar --path ~/d2

## Download sample data

We've created some sample data for you start working on, so that you don't have to wait for your car to be ready. Since he sample data is recorded on another car on another track, it might not be representative for you car. However, it will allow you to get started, and it will provide a good foundation for you to continue training once you get data from you own car.

If you already downloaded the sample data, you can [skip](#Train) this step.

Otherwise, go ahead and download the sample driving run, called *Tubs*, in Donkey:

In [ ]:
from sagemaker import get_execution_role

# Bucket location to get training data
sample_data_location = 's3://jayway-robocar-raw-data/samples'

In [ ]:
# Create a data directory
!mkdir -pv ~/SageMaker/data
!aws s3 cp {sample_data_location}/ore.zip ~/SageMaker/data

In [ ]:
# Unzip to data dir
!unzip -o ~/SageMaker/data/ore.zip -d ~/SageMaker/data

In [ ]:
# check tub size
!du -h ~/SageMaker/data

## Train

### Train a model using the Donkey library

Start training using the `Donkey car` created in `~/d2/`.

The main entrypoint is the `~/d2/manage.py` script, which allows you to start *drive* and *train* sessions. More documentation can be found here:

- [http://docs.donkeycar.com/guide/train_autopilot/](http://docs.donkeycar.com/guide/train_autopilot/)

**Note!** this training is GPU accelerated since we changed dependencies in the installation step. Otherwise, this is a slow operation, around 30 minutes.

In [ ]:
%%time

# Make sure we're in Donkey car root
%cd ~/d2

# Start the training
!python manage.py train --tub='../SageMaker/data/tub_8_18-02-09' --model './models/my-first-model'

There are some interesting information printed during training:

---

`joining the tubs 13630 records together. This could take 0 minutes.`

Training input is created by joining all records in the *Tub* together.

---

`train: 10904, validation: 2726`

The input is split in 2 sets; *training set* and *validation set*, with an 80/20 percent split.

---

`steps_per_epoch 85`

The number of steps per epoch. One **epoch** is one full pass of the training set. The **epoch** is divided up in **batches**, which basically means that the **weights** of the model are updated after each **batch** in the **epoch**. One **step** is one **batch** long.

`steps_per_epoch` is calculated as:

$$
s = \frac{t}{b} = \frac{10904}{128} \approx 85
$$

where $s$ is steps per epoch, $t$ is the length of the training set and $b$ is batch size. Batch size is configurable in the `config.py` file as `BATCH_SIZE`. 

---

```bash
Epoch 1/100
...
85/85 [==============================] - 141s 2s/step - loss: 3.1792 - angle_out_loss: 3.5319 - throttle_out_loss: 0.4266 - val_loss: 2.1385 - val_angle_out_loss: 2.3759 - val_throttle_out_loss: 0.2332
...
```

This is the result of 1 epoch of training. Values doesn't necessarily match yours. A short summary:

|                       | Value       | Description |
| :-                    | :-          | :-          |
| Total epochs          | 100         | The total number of epochs in the training. This is configurable, but not in an easy way |
| Time spent            | 141 seconds | The total time spent training this epoch. Differs depending on instance type and GPU support. |
| Nbr of steps          | 85          | 85 steps per epoch, each step being 128 records long |
| loss                  | 3.1792      | The total training loss after the first epoch. This should decrease for each epoch. |
| angle_out_loss        | 3.5319      | Training loss for angle. |
| throttle_out_loss     | 0.4266      | Training loss for throttle .|
| val_loss              | 2.1385      | The total validation loss after the first epoch. This should decrease for each epoch. |
| val_angle_out_loss    | 2.3759      | Validation loss for angle. |
| val_throttle_out_loss | 0.2332      | Validation loss for throttle. |


### Test the new model

The easiest way to test the model is to use the simulator on your local computer, see [Donkey Tools chapter](./donkey-tools.ipynb#Simulator).

Download the model. It is outputted to `./models`:

In [ ]:
!ls -la ./models

To download the model to you local computer, either use an S3 bucket you've already created in the account, or use the default SageMaker bucket:

In [ ]:
import sagemaker
dl_bucket = sagemaker.Session().default_bucket()

In [ ]:
# push the model to your bucket
!aws s3 cp ./models/my-first-model s3://{dl_bucket}/models/my-first-model

Next, download the model to your computer either manually using the AWS console, or using the CLI if you have it configured:
```bash
aws s3 cp s3://<bucketname>/models/my-first-model .
```

If you've installed the donkey library properly, you should be able to start the model server using:
```bash
donkey sim --model=./models/my-first-model --config ~/d2/config.py
```

If you've installed the simulator properly, you should be able to start it using (on ubuntu):
```bash
./donkey_sim.x86_64
```

Finally, click the **NN Steering over Websockets** button, and the car should start driving using the model (not particulary well though)...

![simulator button](./simulator-connect.png)

## Next

[Digging into the neural network](./donkey-nn.ipynb)